In [6]:
import gymnasium as gym
import time
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Caricamento del modello PPO addestrato
# Assicurati che il percorso al file sia corretto e che il modello esista
model = PPO.load("ppo_HalfCheetah_model")

# Creazione dell'ambiente in modalità vettoriale con DummyVecEnv
# Questo consente la compatibilità con le tecniche di normalizzazione come VecNormalize
render_env = DummyVecEnv([lambda: gym.make("HalfCheetah-v5",
                                           reset_noise_scale=0.013459312664159742,  # Intensità del rumore alla reset
                                           forward_reward_weight=1.4435374113892951,  # Peso della ricompensa per il movimento in avanti
                                           ctrl_cost_weight=0.09129087622076545,  # Peso del costo del controllo (azione)
                                           render_mode='human')])  # Modalità di rendering (può essere 'human' o 'rgb_array')

# Caricamento della normalizzazione dello stato e delle ricompense
# Assicurati che il percorso al file sia corretto e che il file di normalizzazione esista
render_env = VecNormalize.load("vecnormalize_HalfCheetah.pkl", render_env)

# Disabilitiamo l'aggiornamento delle statistiche per la normalizzazione, utile in fase di test
render_env.training = False
render_env.norm_reward = True  # Se True, normalizza anche le ricompense

# Reset dell'ambiente per ottenere l'osservazione iniziale (array numpy)
obs = render_env.reset()

done = False  # Variabile che indica se l'episodio è terminato

# Loop per eseguire la simulazione e il rendering dell'ambiente
while not done:
    action, _ = model.predict(obs)  # Predizione dell'azione ottimale con il modello PPO
    obs, rewards, dones, infos = render_env.step(action)  # Applichiamo l'azione e otteniamo i nuovi dati dall'ambiente
    render_env.render()  # Visualizza l'ambiente a schermo (solo se render_mode='human')
    time.sleep(0.01)  # Piccola pausa per rallentare il rendering e renderlo visibile

    # Gestione del termine dell'episodio
    # `dones` è un array (per compatibilità con più ambienti), prendiamo il primo valore
    done = dones[0]

# Chiusura dell'ambiente per rilasciare le risorse
render_env.close()


/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [4]:
import gymnasium as gym
import time
import numpy as np
import cv2
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Carichiamo il modello PPO addestrato
# Assicurati che il percorso del file sia corretto
model = PPO.load("ppo_HalfCheetah_model")

# Funzione per creare l'ambiente personalizzato con i parametri specificati
def make_env():
    """
    Crea e restituisce l'ambiente "HalfCheetah-v5" dalla libreria Gym con i parametri specificati.

    Questo ambiente viene configurato con valori personalizzati per:
    - reset_noise_scale: livello di rumore al momento del reset dell'ambiente.
    - forward_reward_weight: peso della ricompensa per il movimento in avanti.
    - ctrl_cost_weight: peso del costo del controllo (energia utilizzata per il movimento).
    - render_mode: impostato su 'rgb_array' per ottenere i frame in formato immagine.

    Returns:
        gym.Env: Un'istanza dell'ambiente "HalfCheetah-v5" configurata con i parametri specificati.
    """
    env = gym.make("HalfCheetah-v5",
                   reset_noise_scale=0.013459312664159742,  # Intensità del rumore alla reset
                   forward_reward_weight=1.4435374113892951,  # Peso della ricompensa per il movimento in avanti
                   ctrl_cost_weight=0.09129087622076545,  # Peso del costo del controllo (azione)
                   render_mode='rgb_array')  # Usa 'rgb_array' per ottenere i frame in formato immagine
    return env

# Creiamo un DummyVecEnv per avvolgere l'ambiente, necessario per l'uso di VecNormalize
render_env = DummyVecEnv([make_env])

# Carichiamo la normalizzazione salvata dallo stesso file usato durante l'addestramento
# Assicurati che il percorso al file sia corretto
render_env = VecNormalize.load("vecnormalize_HalfCheetah.pkl", render_env)

# Disattiviamo l'aggiornamento delle statistiche per la normalizzazione (utile in fase di valutazione)
render_env.training = False
render_env.norm_reward = True  # Se True, continua a normalizzare le ricompense

# Reset dell'ambiente per ottenere l'osservazione iniziale (array numpy)
obs = render_env.reset()
done = False  # Variabile che indica se l'episodio è terminato

# Loop per il rendering e la simulazione dell'agente
while not done:
    action, _ = model.predict(obs)  # Predizione dell'azione ottimale con il modello PPO
    obs, reward, done, info = render_env.step(action)  # Esegui l'azione e ottieni i nuovi dati dall'ambiente

    # Ottenere il frame dalla simulazione
    frame = render_env.render()
    if frame is not None:
        cv2.imshow("HalfCheetah Simulation", frame)  # Mostra il frame con OpenCV
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Premere 'q' per uscire dalla simulazione
            break

    time.sleep(0.01)  # Piccola pausa per migliorare la visualizzazione

# Chiusura dell'ambiente e della finestra di visualizzazione
render_env.close()
cv2.destroyAllWindows()


2025-03-04 16:18:51.161 Python[32697:6082046] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-04 16:18:51.161 Python[32697:6082046] +[IMKInputSession subclass]: chose IMKInputSession_Modern
